In [1]:
# library import 
import torch
from torch import nn
import numpy as np
import os
import csv
import math
import pickle
import random
import os

In [2]:
def read_bangla_characters(file_name):
    char_dict = {}
    with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
        csv_lines = csv.DictReader(f)
        for row in csv_lines:
            alphabet = row['alphabet'].strip()
            char_dict[alphabet] = True 
    return char_dict

In [3]:
vowel_path = os.path.join('/home','tiger','Desktop', 'office','bangla-lm','engla', 'resource', 'bn_vowels.csv')
consonant_path = os.path.join('/home','tiger','Desktop', 'office','bangla-lm','engla', 'resource', 'bn_consonants.csv')
kar_path = os.path.join('/home','tiger','Desktop', 'office','bangla-lm','engla', 'resource', 'bn_kars.csv')
bn_vowels = read_bangla_characters(vowel_path)
bn_consonants = read_bangla_characters(consonant_path)
bn_kars = read_bangla_characters(kar_path)
bn_digits = {'০': True, '১': True, '২': True, '৩': True, '৪': True, '৫': True, '৬': True, '৭': True, '৮': True, '৯': True,}

In [64]:
def merge_dicts(list_of_dict):
    global_dict, global_dict2 = {},{}
    cnt = 0
    for l in list_of_dict:
        for key in l:
            if global_dict.get(key) is None:
                global_dict[key] = cnt 
                global_dict2[cnt] = key
                cnt = cnt + 1 
    return global_dict, global_dict2

In [65]:
char2int, int2char = merge_dicts([bn_vowels, bn_consonants, bn_kars, bn_digits])
# nukta
char2int[chr(2509)] = len(char2int)
int2char[len(int2char)] = chr(2509)
# additional space 
char2int[chr(8205)] = len(char2int)
int2char[len(int2char)] = chr(8205)
# space
char2int[" "] = len(char2int)
int2char[len(int2char)] = " "
print(char2int, int2char)
#print(len(char2int))

{'অ': 0, 'আ': 1, 'ই': 2, 'ঈ': 3, 'উ': 4, 'ঊ': 5, 'ঋ': 6, 'এ': 7, 'ঐ': 8, 'ও': 9, 'ঔ': 10, 'ক': 11, 'খ': 12, 'গ': 13, 'ঘ': 14, 'ঙ': 15, 'চ': 16, 'ছ': 17, 'জ': 18, 'ঝ': 19, 'ঞ': 20, 'ট': 21, 'ঠ': 22, 'ড': 23, 'ঢ': 24, 'ণ': 25, 'ত': 26, 'থ': 27, 'দ': 28, 'ধ': 29, 'ন': 30, 'প': 31, 'ফ': 32, 'ব': 33, 'ভ': 34, 'ম': 35, 'য': 36, 'র': 37, 'ল': 38, 'শ': 39, 'ষ': 40, 'স': 41, 'হ': 42, 'ড়': 43, 'ঢ়': 44, 'য়': 45, 'ৎ': 46, 'ং': 47, 'ঃ': 48, 'ঁ': 49, 'া': 50, 'ি': 51, 'ী': 52, 'ু': 53, 'ূ': 54, 'ৃ': 55, 'ে': 56, 'ৈ': 57, 'ো': 58, 'ৌ': 59, '০': 60, '১': 61, '২': 62, '৩': 63, '৪': 64, '৫': 65, '৬': 66, '৭': 67, '৮': 68, '৯': 69, '্': 70, '\u200d': 71, ' ': 72} {0: 'অ', 1: 'আ', 2: 'ই', 3: 'ঈ', 4: 'উ', 5: 'ঊ', 6: 'ঋ', 7: 'এ', 8: 'ঐ', 9: 'ও', 10: 'ঔ', 11: 'ক', 12: 'খ', 13: 'গ', 14: 'ঘ', 15: 'ঙ', 16: 'চ', 17: 'ছ', 18: 'জ', 19: 'ঝ', 20: 'ঞ', 21: 'ট', 22: 'ঠ', 23: 'ড', 24: 'ঢ', 25: 'ণ', 26: 'ত', 27: 'থ', 28: 'দ', 29: 'ধ', 30: 'ন', 31: 'প', 32: 'ফ', 33: 'ব', 34: 'ভ', 35: 'ম', 36: 'য', 37: 'র', 38: 'ল', 39: '

In [6]:
def remove_redundant_space(sentence):
    temp = ""
    for i in range(0, len(sentence)):
        if i-1 >=0 and sentence[i] == " " and temp[-1] == " ":
            continue
        temp = temp + sentence[i]
    temp = temp.strip()
    return temp

In [7]:
def remove_redundant_symbols(sentence):
    temp = ""
    for i in range(0, len(sentence)):
        if sentence[i] == " " or sentence[i] == chr(2509) or sentence[i] == chr(8205) or bn_vowels.get(sentence[i]) is not None or bn_consonants.get(sentence[i])is not None or bn_kars.get(sentence[i]) is not None or bn_digits.get(sentence[i]) is not None:
            if sentence[i] == " ":
                if temp != "" and temp[-1] == ' ':
                    continue
            temp = temp + sentence[i]   
    temp = temp.strip()
    return temp

In [8]:
def read_bn_sentences(file_name, limit_fix=None):
    sentences = []
    with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
        for lines in f:
            line = lines.strip()
            line = remove_redundant_symbols(line)
            if line != "":
                sentences.append(line)
            if len(sentences)%1000 == 0 and len(sentences)>0:
                print(len(sentences))
            if limit_fix is not None and len(sentences) >= limit_fix:
                break
    return sentences

In [9]:
# reading bangla sentences
file_name = os.path.join('/home','tiger','Desktop','office','bangla-data','processed_sentence.txt')
sentences = read_bn_sentences(file_name, limit_fix=100000)
print(len(sentences))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
100000


In [10]:
random.shuffle(sentences)

In [11]:
segment = 100000
complete_db = sentences
sentences = complete_db[:segment]
print(sentences[0:min(10, len(sentences))])

['কুটিরখানা শুন্য রেখে চলে যাব সুদূর আকাশে', 'প্রজাপতির ডানায় নতুন পৃথিবী', 'কিছু বিকৃত চোখের তৃষ্ণা মেটানো এবং চরম ব্যবসায়িক মনোভাব এর মূল লক্ষ্য', 'আমি আমার গল্পের ইঞ্জিনিয়ার ছেলেটির প্রতি চেষ্টা করেও নিষ্ঠুর হতে পারি না', 'আমাদের দেশেরই', 'যারাই শুধু ছুতে পারে আমার বিষাদের ইতিবৃত্ত', 'আমি আর নেই', 'তাই এখানে সেখানে বা থাকি যখন যেখানে', 'বিশ্বব্রহ্মাণ্ডে এই দূরত্ব এত বেশি যে বিজ্ঞানীরা একে মাপতে এককের নাম দিয়েছেন লাইট ইয়ার', 'হাতের চিগারেটটা শেষ করে হাটতে লাগল এতীমখানার উদ্দেশে']


In [181]:
pseudo_sentence = ['আমি ভাত খাই', 'আমি স্কুলে যাই', 'তুমি কি করো']

In [182]:
sentences = pseudo_sentence
print(sentences)

['আমি ভাত খাই', 'আমি স্কুলে যাই', 'তুমি কি করো']


In [12]:
def find_mean_sentence_length(sentences):
    mean = 0
    for i in range(0, len(sentences)):
        sentences[i] = sentences[i].strip()
        mean = mean + len(sentences[i])
    mean = mean / (1.0 * len(sentences))   
    mean = math.floor(mean)
    print("mean {}".format(mean))
    return mean

In [13]:
def check_too_long_words(sentence):
    words = sentence.split(" ")
    s = []
    for w in words:
        if len(w) >= 25:
            return False 
    return True

In [14]:
def sentence_max_length_process(sentences, start_max_length):
    i = 0 
    save = []
    temp_ml = 0
    idx_save = None
    while i < len(sentences):
        if i%10000 == 0:
            print(i)
        verdict = check_too_long_words(sentences[i])
        # print(verdict, sentences[i])
        if verdict is False:
            print(verdict, sentences[i])
            i = i + 1
            continue
        if len(sentences[i]) <= start_max_length:
            save.append(sentences[i])
            temp_ml = max(temp_ml, len(save[-1]))
            if temp_ml == len(sentences[i]):
                idx_save = len(save)-1
            i = i + 1
        else:
            sentences[i] = sentences[i].strip()
            words = sentences[i].split(" ")
            divide = [""]
            for j in range(0, len(words)):
                if len(divide[-1]) + len(words[j]) > start_max_length:
                    divide[-1] = divide[-1].strip()
                    divide.append("")
                    divide[-1] = words[j]
                else:
                    divide[-1] = divide[-1] + " " + words[j]
            for j in range(0, len(divide)):
                save.append(divide[j])
                temp_ml = max(temp_ml, len(divide[j]))
                if temp_ml == len(save[-1]):
                    idx_save = len(save)-1
            i = i + 1
    print(temp_ml, idx_save, save[idx_save])
    return save, temp_ml

In [15]:
def find_largest_length(sentences, sentence_avg_length):
    max_length = 0 
    idx = 0
    for i in range(0,len(sentences)):
        if max_length < len(sentences[i]):
            max_length = len(sentences[i])
            idx = i
    print(max_length, idx, sentences[idx], len(sentences[idx]))
    return max_length

In [16]:
def delete_sentence(sentences, max_length, allowed_gap):
    i = 0
    save = []
    while i<len(sentences):
        # print(sentences[i])
        if len(sentences[i]) < (max_length-allowed_gap):
            i=i+1
        else:
            save.append(sentences[i])
            i=i+1
    return save

In [17]:
print(find_mean_sentence_length(sentences))

mean 44
44


In [66]:
sequence_max_length = 15

In [67]:
updated_sentences,sequence_max_length = sentence_max_length_process(sentences, sequence_max_length)
print("length = ",len(updated_sentences), sequence_max_length)

0
10000
False আবার আমরা দেশ প্রেমিকলেখকবিজ্ঞানীগণিতবিদ ও শিল্পী খুজে পাব
20000
False আর এই দৃশ্য দেখার পর আমার একটা কথায় মনে হয় রুশো মন্টেসরীফ্রয়েবেলরবীন্দ্রনাথ এদের শিক্ষা দর্শন আজ ফিকে হয়ে যাচ্ছে
30000
False টেলিভিশনপত্রিকাফেসবুকব্লগ ও
40000
False শিল্প সাহিত্যের জ্ঞান ছড়িয়ে দেবার জন্য বিশ্ববিদ্যালয়পাঠাগারস্কুলকলেজ ভিত্তিক নানান প্রচেষ্টা হাতে নেয়া যেতে পারে
50000
False কারণ তাঁর চেষ্ঠাপ্রচেষ্ঠাকর্মদক্ষতা ও সচেতনতায় দেশের সবচেয়ে বাল্যবিয়ে প্রবণ উপজেলা সরাইল আজ নিরানব্বই ভাগ বাল্যবিয়ে মুক্ত উপজেলা
False দূর থেকে পড়তে না জানলে পুলিশছিনতাইকারীচোরবারবনিতাদের অন্যায় আক্রমনের শিকার হতে পারে যে কোনো সময়
60000
False নৈতিক শিক্ষাই একজন মানুষকে প্রতারকভন্ডছিনতাইকারীধর্ষকঘুষখোর ও যে কোন দূর্নীতিপরায়ন হয়ে উঠতে বাধা দেয়
70000
80000
False দিনদিন আমরা হয়ে উঠছি ধর্ষকপ্রতারকঘুষখোরছিনতাইকারীসিরিয়াল কিলারবোমাবাজ ভন্ড নেতা উগ্রপন্থী চরমপন্থীসাম্প্রদায়িক কিন্তু কেন এই পরিণতি আমাদের আমাদের তো ছিল সুন্দর ইতিহাস
90000
False হাইহ্যালোআন্তরিকতাভালোবাসা সবার সাথে আদান প্রদান শেষে আপুকে নিয়ে উদ্যানে ঢুকলাম শাওন

In [68]:
updated_sentences = delete_sentence(updated_sentences, sequence_max_length, allowed_gap=10)
print(len(updated_sentences), sequence_max_length)

132456 24


In [21]:
def padding_sentences(sentences, start_max_length):
    for i in range(0, len(sentences)):
        while len(sentences[i]) < start_max_length:
            sentences[i] = sentences[i]+ " "
        if len(updated_sentences[i]) != start_max_length:
            print("violated ", len(updated_sentences[i]), start_max_length)
    return sentences

In [69]:
updated_sentences = padding_sentences(updated_sentences, sequence_max_length)

In [23]:
def sanity_check_seq_len(updated_sentences, sequence_max_length):
    l = sequence_max_length
    for i in range(0, len(updated_sentences)):
        if len(updated_sentences[i]) != sequence_max_length:
            print(len(updated_sentences[i]))
        l = min(l, len(updated_sentences[i]))
    print(l)

In [70]:
sanity_check_seq_len(updated_sentences, sequence_max_length)

24


In [71]:
def input_target_make(sentences):
    input_sen, target_sen = [], []
    for i in range(0, len(sentences)):
        input_sen.append(sentences[i][:-1])
        target_sen.append(sentences[i][1:])
    return input_sen, target_sen
def sanity_test_input_target(input_sen, target_sen):
    for i in range(0,min(len(input_sen),50)):
        print(input_sen[i], target_sen[i])

In [72]:
input_sen, target_sen = input_target_make(updated_sentences)
sanity_test_input_target(input_sen, target_sen)

কুটিরখানা শুন্য         ুটিরখানা শুন্য         
প্রজাপতির ডানায়         ্রজাপতির ডানায়         
মেটানো এবং চরম          েটানো এবং চরম          
ব্যবসায়িক মনোভাব        ্যবসায়িক মনোভাব        
আমি আমার গল্পের         মি আমার গল্পের         
নিষ্ঠুর হতে পারি        িষ্ঠুর হতে পারি        
যারাই শুধু ছুতে         ারাই শুধু ছুতে         
বিষাদের ইতিবৃত্ত        িষাদের ইতিবৃত্ত        
সেখানে বা থাকি          েখানে বা থাকি          
বিশ্বব্রহ্মাণ্ডে        িশ্বব্রহ্মাণ্ডে        
বিজ্ঞানীরা একে          িজ্ঞানীরা একে          
মাপতে এককের নাম         াপতে এককের নাম         
হাতের চিগারেটটা         াতের চিগারেটটা         
লাগল এতীমখানার          াগল এতীমখানার          
বাতাসে বক্ষ ভরি         াতাসে বক্ষ ভরি         
স্থানগুলোর ছবি          ্থানগুলোর ছবি          
একটা করে পোষ্ট          কটা করে পোষ্ট          
কর্মসূত্রে তিনি         র্মসূত্রে তিনি         
একসাইটেট স্বপন          কসাইটেট স্বপন          
ক্ষেত ধরেই তাদের        ্ষেত ধরেই তাদের        
দিল্লিকা লাড্ডু         িল্লিকা লাড্ডু  

In [27]:
print(len(input_sen), len(target_sen))

131950 131950


In [29]:
def sequence_to_int_conversion(sentence):
    t = []
    # print(char2int)
    for i in range(0, len(sentence)):
        if char2int.get(sentence[i]) is not None:
            t.append(char2int.get(sentence[i]))
        else:
            t.append(len(char2int))
    return t

In [73]:
input_sen_bit, target_sen_bit, = [],[]
for i in range(0, len(input_sen)):
    input_sen_bit.append(sequence_to_int_conversion(input_sen[i]))
    target_sen_bit.append(sequence_to_int_conversion(target_sen[i]))
    # print(input_sen[i], input_sen_bit[-1])
    if i%1000 == 0:
        print(i)
print(input_sen_bit[0:10])
print(input_sen[0], input_sen_bit[0], len(input_sen_bit[0]))
print(target_sen[0], target_sen_bit[0], len(target_sen_bit[0]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
[[11, 53, 21, 51, 37, 12, 50, 30, 50, 72, 39, 53, 30, 70, 36, 72, 72, 72, 72, 72, 72, 72, 72], [31, 70, 37, 18, 50, 31, 26, 51, 37, 72, 23, 50, 30, 50, 45, 72, 72, 72, 72, 72, 72, 72

In [5]:
def save_into_pickle(objs):
    # Saving the objects:
    with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(objs, f)
        
def load_from_pickle():
    # Getting back the objects:
    with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        objs = pickle.load(f)
    return objs

In [7]:
test = [1,2,3]
save_into_pickle([test])
objs = load_from_pickle()
print(objs)

[[1, 2, 3]]


In [74]:
dict_size = len(char2int)
seq_len = sequence_max_length-1
batch_size = len(input_sen)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    print(features.shape)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        # print(i, sequence[i], len(sequence[i]))
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [75]:
input_sen_hot_encode = one_hot_encode(input_sen_bit, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_sen_hot_encode.shape))

(132456, 23, 73)
Input shape: (132456, 23, 73) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [125]:
v = {}
for i in range(0,len(target_sen_bit)):
    if len(target_sen_bit[i]) not in v:
        v[len(target_sen_bit[i])] = 0
    v[len(target_sen_bit[i])] = v[len(target_sen_bit[i])] + 1
print(v)

{30: 131950}


In [76]:
input_sen_hot_encode = torch.from_numpy(input_sen_hot_encode)
target_sen_hot_encode = torch.Tensor(target_sen_bit)

In [34]:
print(input_sen_hot_encode.shape)
print(target_sen_hot_encode.shape)

torch.Size([131950, 30, 73])
torch.Size([131950, 30])


In [77]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [78]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [79]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=20, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [38]:
def save_chekcpoint(epoch, model_path, loss):
    EPOCH = epoch
    PATH = model_path
    LOSS = loss

    torch.save({
                'epoch': EPOCH,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': LOSS,
                }, PATH)


In [94]:
def load_checkpoint(PATH):
    if os.path.exists(PATH) is True:
        model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=20, n_layers=1)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print("checkpoint loaded")
        print(checkpoint['model_state_dict'])
        criterion = nn.CrossEntropyLoss()
        return epoch, loss, model, optimizer
    else:
        return None, None, None, None

In [81]:
already_loss = None

In [104]:
already_epoch, already_loss, model_, optimizer_ = load_checkpoint('model.pt')
if model_ is not None:
    model = model_
    optimizer = optimizer_
    model.train()
print(already_epoch, already_loss)

checkpoint loaded
OrderedDict([('rnn.weight_ih_l0', tensor([[ 0.7722,  0.0673,  0.1678,  ...,  0.0988, -0.0139, -0.0338],
        [ 0.5495,  1.2674, -0.3873,  ...,  0.4859, -0.1725, -0.1429],
        [ 0.6228,  0.4732, -0.4486,  ...,  1.6023, -0.0264,  0.4706],
        ...,
        [ 1.5699,  0.9656,  0.0285,  ...,  0.9069, -0.0911, -0.3848],
        [-0.4140, -0.0913,  0.1141,  ...,  0.2426,  0.1132, -0.3177],
        [-0.0101, -0.9493, -0.1819,  ..., -0.2745,  0.0181,  0.4105]])), ('rnn.weight_hh_l0', tensor([[ 7.8423e-01,  1.3394e-01,  4.4591e-02, -2.9729e-01, -3.4106e-02,
          1.7427e-01,  1.0950e-01, -1.3249e-01,  4.1050e-01,  2.3295e-01,
          6.4822e-01, -1.8699e-01,  6.9288e-02,  6.9375e-01,  3.6745e-01,
         -5.2909e-02, -1.9706e-01,  1.7563e-01,  3.0234e-01, -1.6617e-01],
        [ 1.2958e-01, -4.5851e-02, -2.5513e-01, -1.8209e-01,  1.7162e-01,
         -2.4502e-01, -3.3332e-01, -2.7899e-02,  9.7448e-04,  3.3103e-01,
          4.9150e-04,  4.8055e-02, -2.1716e-01

In [103]:
# Training Run
input_sen_hot_encode = input_sen_hot_encode.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_sen_hot_encode)
    output = output.to(device)
    target_sen_hot_encode = target_sen_hot_encode.to(device)
    loss = criterion(output, target_sen_hot_encode.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 1.5670
Epoch: 20/100............. Loss: 1.5613
Epoch: 30/100............. Loss: 1.5575
Epoch: 40/100............. Loss: 1.5571
Epoch: 50/100............. Loss: 1.5546
Epoch: 60/100............. Loss: 1.5518
Epoch: 70/100............. Loss: 1.5821
Epoch: 80/100............. Loss: 1.5608
Epoch: 90/100............. Loss: 1.5941
Epoch: 100/100............. Loss: 1.6093


In [101]:
if already_loss is not None and loss.item() < already_loss:
    save_chekcpoint(already_epoch+n_epochs, 'model.pt', loss.item())
if already_loss is None:
    already_loss, already_epoch = loss.item(), n_epochs
    save_chekcpoint(already_epoch, 'model.pt', loss.item())

In [54]:
def get_best_two_characters(tensor_array):
    # the output of the line below is a numpy array
    if is_cuda is True:
        numpy_arr = tensor_array.gpu().detach().numpy() 
    else:
        numpy_arr = tensor_array.cpu().detach().numpy()
    best_two = [(0, numpy_arr[0])]
    if numpy_arr[1] > best_two[0][1]:
        best_two.append(best_two[0])
        best_two[0] = (1, numpy_arr[1])
    else:
        best_two.append((1, numpy_arr[1]))
    for i in range(2, len(numpy_arr)):
        if numpy_arr[i] > best_two[0][1]:
            best_two[1] = best_two[0]
            best_two[0] = (i, numpy_arr[i])
        elif numpy_arr[i] > best_two[1][1]:
            best_two[1] = (i, numpy_arr[i])
    return best_two

In [51]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)
    # print("out  = ", out)
    print(out.shape)
    # print(out[-1])

    prob = nn.functional.softmax(out[-1], dim=0).data
    print("pob = ", prob)
    print(prob.shape)
    best_two = get_best_two_characters(prob)
    print(best_two)
    # Taking the class with the highest probability score from the output
    """
    char_ind = torch.max(prob, dim=0)[1].item()
    print(char_ind)
    """

    return int2char[best_two[0][0]],int2char[best_two[1][0]], hidden

In [105]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char1, char2, h = predict(model, chars)
        if chars[-1] == " ":
            chars.append(char2)
        else:
            chars.append(char1)
        print(''.join(chars))
    return ''.join(chars)

In [107]:
sample(model, 20, 'একসাথে মেয়ের')

(1, 12, 73)
torch.Size([12, 73])
pob =  tensor([3.8787e-05, 2.4160e-04, 1.9326e-03, 9.5021e-06, 2.5860e-04, 2.9183e-06,
        5.2363e-06, 1.5479e-04, 1.7355e-05, 3.7140e-03, 4.5688e-06, 4.0426e-03,
        3.3022e-04, 1.5019e-03, 1.0798e-04, 1.9359e-04, 7.7442e-04, 2.7498e-03,
        7.6874e-04, 3.9358e-05, 4.4971e-05, 2.7183e-03, 1.4730e-04, 8.8566e-05,
        1.8966e-05, 4.0042e-03, 1.0289e-02, 4.8859e-04, 2.0788e-03, 2.3806e-04,
        3.9683e-03, 1.4308e-03, 3.2525e-04, 3.2364e-03, 4.7940e-04, 2.1157e-03,
        3.1162e-04, 3.4703e-03, 2.1318e-03, 1.0963e-03, 2.9008e-04, 9.8247e-04,
        1.1392e-03, 6.1691e-05, 8.0926e-06, 1.9026e-03, 1.1884e-04, 2.1119e-04,
        3.8669e-05, 3.1507e-05, 3.9811e-02, 1.8663e-02, 8.2764e-03, 2.8553e-03,
        1.2143e-04, 6.0557e-05, 5.4721e-02, 8.4830e-05, 9.8431e-03, 3.9843e-05,
        2.1747e-06, 6.0907e-06, 3.2448e-06, 2.1916e-06, 2.4510e-06, 3.0319e-06,
        2.2950e-06, 1.8204e-06, 9.1347e-06, 3.2095e-06, 7.8892e-03, 2.9919e-06,


'একসাথে মেয়ের কর ক ন '

In [179]:
print(sentences[0])

তার মস্তিষ্কটাকে যেন প্লেগ্রাউন্ড বানিয়ে ইচ্ছেমতো ছোটাছুটি করছে শব্দেরা
